In [1]:
# Install required libraries in Colab
#!pip install folium geopandas shapely pandas
#!pip install --upgrade folium branca

import pandas as pd
import folium
from folium import Choropleth
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
import numpy as np
from google.colab import files
from shapely.geometry import Point, MultiLineString, LineString
from shapely.ops import unary_union, nearest_points

# Initialize ward_connectivity as a dictionary
ward_connectivity = {}

# Load GeoJSON datasets
bike_parking = gpd.read_file('Bike_Parking.geojson')
bikeways = gpd.read_file('Bikeways.geojson')
bike_share_hubs = gpd.read_file('Hamilton_Bike_Share_Incorporated_Hubs.geojson')
ward_boundaries = gpd.read_file('Ward_Boundaries.geojson')
city_population = gpd.read_file('City_Growth_Targets_Population.geojson')

# Clean datasets
bike_parking = bike_parking.dropna(subset=['geometry'])
bike_share_hubs = bike_share_hubs.dropna(subset=['geometry'])
bikeways = bikeways.dropna(subset=['geometry'])
ward_boundaries = ward_boundaries.dropna(subset=['geometry'])

# Ensure all datasets are in EPSG:4326 (WGS84) for Folium
bike_parking = bike_parking.to_crs(epsg=4326)
bike_share_hubs = bike_share_hubs.to_crs(epsg=4326)
bikeways = bikeways.to_crs(epsg=4326)
ward_boundaries = ward_boundaries.to_crs(epsg=4326)

# Convert WARD columns to strings to ensure consistency
bike_parking['WARD'] = bike_parking['WARD'].astype(str).replace('None', np.nan)
bikeways['WARD'] = bikeways['WARD'].astype(str).replace('None', np.nan)
ward_boundaries['WARD'] = ward_boundaries['WARD'].astype(str).replace('None', np.nan)

# Create a union of ward boundaries to define the city extent
city_boundary = unary_union(ward_boundaries.geometry)

# Clip bike routes to the city boundary
bikeways_clipped = bikeways.copy()
bikeways_clipped['geometry'] = bikeways_clipped.geometry.intersection(city_boundary)
# Remove any empty geometries after clipping
bikeways_clipped = bikeways_clipped[~bikeways_clipped.geometry.is_empty]

# Spatially join bike_share_hubs with ward_boundaries to assign wards
bike_share_hubs = gpd.sjoin(bike_share_hubs, ward_boundaries, how='left', predicate='intersects')
bike_share_hubs['WARD'] = bike_share_hubs['WARD'].astype(str).replace('nan', np.nan)

# Remove outliers in bike parking (e.g., in the lake)
bike_parking = gpd.sjoin(bike_parking, ward_boundaries, how='inner', predicate='intersects')
bike_parking['WARD'] = bike_parking['WARD_left']  # Restore the original WARD from bike_parking
bike_parking = bike_parking.drop(columns=['WARD_left', 'WARD_right'])  # Drop the duplicate columns

# Calculate population growth and estimate 2025 population
actual_population = city_population[city_population['TOTAL_POPULATION'].notna()].copy()
actual_population = actual_population.sort_values('YEAR')
actual_population['GROWTH_RATE'] = actual_population['TOTAL_POPULATION'].pct_change() * 100
average_growth_rate = actual_population['GROWTH_RATE'].mean()
pop_2021 = actual_population[actual_population['YEAR'] == '2021']['TOTAL_POPULATION'].iloc[0]
years_to_2025 = 2025 - 2021
pop_2025 = pop_2021 * (1 + average_growth_rate / 100) ** years_to_2025
pop_2030_forecast = city_population[city_population['YEAR'] == '2030']['PROV_GROWTH_PLAN_FORECAST'].iloc[0]

# Center map on Hamilton, Ontario
hamilton_center = [43.2557, -79.8711]  # Approx center of Hamilton
m = folium.Map(location=hamilton_center, zoom_start=12, tiles='CartoDB positron')

# 1. Bike Routes Layer (Single Color: Blue)
status_styles = {
    'Existing': {'dashArray': '1, 0'},  # Solid line
    'Upgrade': {'dashArray': '5, 5'}    # Dashed line
}
comfort_styles = {
    'Comfort': {'weight': 5, 'opacity': 1.0},
    'Cautious or hilly': {'weight': 3, 'opacity': 0.6},
    'Busy or steep': {'weight': 2, 'opacity': 0.4},
    None: {'weight': 3, 'opacity': 0.8}
}

route_layer = folium.FeatureGroup(name="Bike Routes")
for _, row in bikeways_clipped.iterrows():
    if isinstance(row.geometry, LineString):
        coords = [(point[1], point[0]) for point in row.geometry.coords]
        folium.PolyLine(
            locations=coords,
            color='blue',
            weight=comfort_styles.get(row.get('COMFORT_RATING'), comfort_styles[None])['weight'],
            opacity=comfort_styles.get(row.get('COMFORT_RATING'), comfort_styles[None])['opacity'],
            dash_array=status_styles.get(row['STATUS'], {'dashArray': '1, 0'})['dashArray'],
            popup=f"Type: {row['TYPE']}<br>Status: {row['STATUS']}<br>Comfort: {row.get('COMFORT_RATING', 'Unknown')}<br>Length: {row['Shape__Length']:.2f}m<br>Ward: {row['WARD']}"
        ).add_to(route_layer)
    elif isinstance(row.geometry, MultiLineString):
        for line in row.geometry.geoms:
            coords = [(point[1], point[0]) for point in line.coords]
            folium.PolyLine(
                locations=coords,
                color='blue',
                weight=comfort_styles.get(row.get('COMFORT_RATING'), comfort_styles[None])['weight'],
                opacity=comfort_styles.get(row.get('COMFORT_RATING'), comfort_styles[None])['opacity'],
                dash_array=status_styles.get(row['STATUS'], {'dashArray': '1, 0'})['dashArray'],
                popup=f"Type: {row['TYPE']}<br>Status: {row['STATUS']}<br>Comfort: {row.get('COMFORT_RATING', 'Unknown')}<br>Length: {row['Shape__Length']:.2f}m<br>Ward: {row['WARD']}"
            ).add_to(route_layer)
route_layer.add_to(m)

# 2. Bike Parking Cluster (Color-coded by Capacity)
parking_cluster = MarkerCluster(name="Bike Parking").add_to(m)
for _, row in bike_parking.iterrows():
    capacity = row['TOTAL_CAPACITY']
    color = 'green' if capacity >= 10 else 'orange' if capacity >= 5 else 'red'
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=f"Location: {row['LOCATION_NAME']}<br>Capacity: {capacity}<br>Ward: {row['WARD']}",
        icon=folium.Icon(color=color, icon='bicycle', prefix='fa')
    ).add_to(parking_cluster)

# 3. Bike Share Hubs with Capacity Circles
share_hubs_layer = folium.FeatureGroup(name="Bike Share Hubs")
for _, row in bike_share_hubs.iterrows():
    ward_info = f"Ward: {row['WARD']}" if pd.notna(row['WARD']) else "Ward: Unknown"
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=row['RACKS_AMOUNT'] * 2,
        color='purple',
        fill=True,
        fill_color='purple',
        fill_opacity=0.6,
        popup=f"Station: {row['NAME']}<br>Racks: {row['RACKS_AMOUNT']}<br>Available: {row['AVAILABLE_BIKES']}<br>{ward_info}"
    ).add_to(share_hubs_layer)
share_hubs_layer.add_to(m)

# 4. Heatmap of Bike Infrastructure Density
heat_data = [[row.geometry.y, row.geometry.x] for _, row in pd.concat([bike_parking, bike_share_hubs]).iterrows()]
HeatMap(heat_data, radius=15, blur=20, name="Infrastructure Density").add_to(m)

# 5. Gap Analysis: Identify Parking and Hubs Not Near Routes
gdf_routes_buffer = bikeways_clipped.to_crs("EPSG:32617").buffer(500).to_crs("EPSG:4326")  # 500m buffer
bike_parking['near_route'] = bike_parking.geometry.apply(lambda x: any(gdf_routes_buffer.contains(x)))
bike_share_hubs['near_route'] = bike_share_hubs.geometry.apply(lambda x: any(gdf_routes_buffer.contains(x)))

# Uncovered Parking (Parking spots not within 500m of a bike route)
uncovered_parking = bike_parking[~bike_parking['near_route']]
uncovered_parking_cluster = MarkerCluster(name="Uncovered Bike Parking (Not Near Routes)").add_to(m)
for _, row in uncovered_parking.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=f"Uncovered Bike Parking (Not within 500m of a bike route)<br>Location: {row['LOCATION_NAME']}<br>Capacity: {row['TOTAL_CAPACITY']}<br>Ward: {row['WARD']}",
        icon=folium.Icon(color='orange', icon='exclamation-triangle', prefix='fa')
    ).add_to(uncovered_parking_cluster)

# Uncovered Hubs (Bike share hubs not within 500m of a bike route)
uncovered_hubs = bike_share_hubs[~bike_share_hubs['near_route']]
uncovered_hubs_cluster = MarkerCluster(name="Uncovered Bike Share Hubs (Not Near Routes)").add_to(m)
for _, row in uncovered_hubs.iterrows():
    ward_info = f"Ward: {row['WARD']}" if pd.notna(row['WARD']) else "Ward: Unknown"
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        popup=f"Uncovered Bike Share Hub (Not within 500m of a bike route)<br>Station: {row['NAME']}<br>Racks: {row['RACKS_AMOUNT']}<br>{ward_info}"
    ).add_to(uncovered_hubs_cluster)

# 6. Ward-Level Statistics and Connectivity Score
ward_stats = []
for ward in bike_parking['WARD'].unique():
    if pd.isna(ward) or ward == 'nan':
        continue
    ward_parking = bike_parking[bike_parking['WARD'] == ward]
    ward_hubs = bike_share_hubs[(bike_share_hubs['WARD'].notna()) & (bike_share_hubs['WARD'] != 'nan') & (bike_share_hubs['WARD'] == ward)]
    ward_routes = bikeways_clipped[(bikeways_clipped['WARD'].notna()) & (bikeways_clipped['WARD'] != 'nan') & (bikeways_clipped['WARD'] == ward)]
    ward_boundary = ward_boundaries[ward_boundaries['WARD'] == ward]

    # Bike Parking Stats
    total_parking_spots = len(ward_parking)
    total_parking_capacity = ward_parking['TOTAL_CAPACITY'].sum() if total_parking_spots > 0 else 0
    total_hubs = len(ward_hubs)
    total_hub_racks = ward_hubs['RACKS_AMOUNT'].sum() if total_hubs > 0 else 0
    uncovered_parking_count = len(ward_parking[~ward_parking['near_route']])
    uncovered_hubs_count = len(ward_hubs[~ward_hubs['near_route']])

    # Calculate bikes per capita (total bikes = parking capacity + hub racks)
    total_bikes = total_parking_capacity + total_hub_racks
    num_wards = len(bike_parking['WARD'].unique())
    ward_population_2025 = pop_2025 / num_wards
    bikes_per_capita_2025 = total_bikes / ward_population_2025 if ward_population_2025 > 0 else 0

    # Calculate ward area (in square meters) for infrastructure density
    ward_area = ward_boundary.to_crs(epsg=32617)['geometry'].area.sum() if not ward_boundary.empty else 1  # Avoid division by zero
    bikes_per_km2 = total_bikes / (ward_area / 1_000_000) if ward_area > 0 else 0  # Bikes per square kilometer

    # Connectivity Score (Normalized to 0-100)
    parking_near_routes = ward_parking['near_route'].sum() / len(ward_parking) if len(ward_parking) > 0 else 0
    hubs_near_routes = ward_hubs['near_route'].sum() / len(ward_hubs) if len(ward_hubs) > 0 else 0
    route_length = ward_routes['geometry'].length.sum()
    comfortable_routes = ward_routes[ward_routes['COMFORT_RATING'] == 'Comfort']['geometry'].length.sum()
    route_comfort_coverage = comfortable_routes / route_length if route_length > 0 else 0
    route_length_normalized = min(route_length / 100_000, 1)
    bikes_per_capita_normalized = min(bikes_per_capita_2025 / 0.1, 1)
    bikes_per_km2_normalized = min(bikes_per_km2 / 1000, 1)
    score = (0.2 * parking_near_routes + 0.2 * hubs_near_routes + 0.2 * route_length_normalized + 0.2 * route_comfort_coverage + 0.1 * bikes_per_capita_normalized + 0.1 * bikes_per_km2_normalized) * 100
    ward_connectivity[ward] = score

    # Convert route length to kilometers
    route_length_km = route_length / 1000

    # Ward Statistics Dictionary
    ward_stats.append({
        'Ward': ward,
        'Total_Parking_Spots': total_parking_spots,
        'Total_Parking_Capacity': total_parking_capacity,
        'Total_Hubs': total_hubs,
        'Total_Hub_Racks': total_hub_racks,
        'Uncovered_Parking': uncovered_parking_count,
        'Uncovered_Hubs': uncovered_hubs_count,
        'Bikes_Per_Capita_2025': bikes_per_capita_2025,
        'Bikes_Per_Sq_Km': bikes_per_km2,
        'Connectivity_Score': score,
        'Comfortable_Route_Coverage': route_comfort_coverage * 100,
        'Total_Route_Length_km': route_length_km
    })

# Convert ward stats to a DataFrame for display
ward_stats_df = pd.DataFrame(ward_stats)
print("\nWard-by-Ward Bike Infrastructure Statistics:\n")
print(ward_stats_df.sort_values('Ward').to_string(index=False))

# 7. Ward Connectivity Visualization using Choropleth with Ward Numbers
ward_connectivity_df = pd.DataFrame.from_dict(ward_connectivity, orient='index', columns=['Connectivity_Score'])
ward_connectivity_df['Ward'] = ward_connectivity_df.index.astype(str)
ward_boundaries['WARD'] = ward_boundaries['WARD'].astype(str)
Choropleth(
    geo_data=ward_boundaries,
    name='Ward Connectivity (Choropleth)',
    data=ward_connectivity_df,
    columns=['Ward', 'Connectivity_Score'],
    key_on='feature.properties.WARD',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Connectivity Score (Higher = Better)',
    highlight=True
).add_to(m)

# Add ward numbers as labels
ward_labels_layer = folium.FeatureGroup(name="Ward Labels")
for _, ward in ward_boundaries.iterrows():
    ward_id = ward['WARD']
    # Calculate the centroid of the ward for placing the label
    centroid = ward.geometry.centroid
    folium.Marker(
        location=[centroid.y, centroid.x],
        icon=folium.DivIcon(
            html=f'<div style="font-size: 12pt; font-weight: bold; color: black; text-shadow: 1px 1px 2px white;">{ward_id}</div>'
        )
    ).add_to(ward_labels_layer)
ward_labels_layer.add_to(m)

# 8. Add Interactive Ward Popups with Statistics
ward_stats_df.set_index('Ward', inplace=True)
ward_popup_layer = folium.FeatureGroup(name="Ward Statistics")
for _, ward in ward_boundaries.iterrows():
    ward_id = ward['WARD']
    if ward_id in ward_stats_df.index:
        stats = ward_stats_df.loc[ward_id]
        popup_text = (
            f"<b>Ward {ward_id}</b><br>"
            f"Connectivity Score: {stats['Connectivity_Score']:.2f}/100<br>"
            f"Total Parking Spots: {int(stats['Total_Parking_Spots'])}<br>"
            f"Total Parking Capacity: {int(stats['Total_Parking_Capacity'])}<br>"
            f"Total Hubs: {int(stats['Total_Hubs'])}<br>"
            f"Total Hub Racks: {int(stats['Total_Hub_Racks'])}<br>"
            f"Uncovered Parking: {int(stats['Uncovered_Parking'])}<br>"
            f"Uncovered Hubs: {int(stats['Uncovered_Hubs'])}<br>"
            f"Bikes Per Capita (2025): {stats['Bikes_Per_Capita_2025']:.4f}<br>"
            f"Bikes Per Sq Km: {stats['Bikes_Per_Sq_Km']:.2f}<br>"
            f"Comfortable Route Coverage: {stats['Comfortable_Route_Coverage']:.2f}%<br>"
            f"Total Route Length: {stats['Total_Route_Length_km']:.2f}km"
        )
        folium.GeoJson(
            ward.geometry.__geo_interface__,
            style_function=lambda x: {'color': 'black', 'weight': 1, 'dashArray': '5, 5', 'fillOpacity': 0},
            popup=folium.Popup(popup_text, max_width=300)
        ).add_to(ward_popup_layer)
ward_popup_layer.add_to(m)

# 9. Development Opportunities: Create Bike Routes from Uncovered Parking/Hubs to Nearest Bike Route
development_layer = folium.FeatureGroup(name="Development Opportunities (Proposed Bike Routes)")
# Combine uncovered parking and hubs into a single GeoDataFrame for processing
uncovered_locations = pd.concat([uncovered_parking, uncovered_hubs], ignore_index=True)
# Create a union of all existing bike routes for finding nearest points
bike_routes_union = unary_union(bikeways_clipped.geometry)

for _, loc in uncovered_locations.iterrows():
    loc_point = loc.geometry
    # Find the nearest point on the bike route network
    nearest_route_point = nearest_points(loc_point, bike_routes_union)[1]
    # Create a straight-line route from the location to the nearest route point
    new_route = LineString([loc_point, nearest_route_point])
    # Ensure the new route is within the city boundary
    new_route_clipped = new_route.intersection(city_boundary)
    if new_route_clipped.is_empty:
        continue
    # Convert to coordinates for Folium
    if isinstance(new_route_clipped, LineString):
        coords = [(point[1], point[0]) for point in new_route_clipped.coords]
        folium.PolyLine(
            locations=coords,
            color='purple',
            weight=3,
            dash_array='5, 5',  # Dashed line to indicate proposed route
            popup=f"Proposed Bike Route<br>From: {'Parking' if 'LOCATION_NAME' in loc else 'Hub'}<br>Ward: {loc['WARD']}"
        ).add_to(development_layer)
    elif isinstance(new_route_clipped, MultiLineString):
        for line in new_route_clipped.geoms:
            coords = [(point[1], point[0]) for point in line.coords]
            folium.PolyLine(
                locations=coords,
                color='purple',
                weight=3,
                dash_array='5, 5',
                popup=f"Proposed Bike Route<br>From: {'Parking' if 'LOCATION_NAME' in loc else 'Hub'}<br>Ward: {loc['WARD']}"
            ).add_to(development_layer)

development_layer.add_to(m)

# 10. Add Layers Control and Title
folium.LayerControl().add_to(m)
m.get_root().html.add_child(folium.Element(
    "<h3 style='text-align:center;'>Hamilton Bike Infrastructure Analysis</h3>"
    "<p style='text-align:center;'>Promoting Cycling as Alternative Transport in Hamilton, Ontario</p>"
    f"<p style='text-align:center;'>Estimated Population 2025: {pop_2025:.0f} | Forecasted Population 2030: {pop_2030_forecast:.0f}</p>"
))

# Save and display the map
m.save('hamilton_bike_analysis.html')
from IPython.display import IFrame
IFrame(src='hamilton_bike_analysis.html', width=700, height=600)

# Download the map
files.download('hamilton_bike_analysis.html')

<ipython-input-1-36db1239dee4>:202: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  route_length = ward_routes['geometry'].length.sum()
<ipython-input-1-36db1239dee4>:203: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  comfortable_routes = ward_routes[ward_routes['COMFORT_RATING'] == 'Comfort']['geometry'].length.sum()
<ipython-input-1-36db1239dee4>:202: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  route_length = ward_routes['geometry'].length.sum()
<ipython-input-1-36db1239dee4>:203: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use '


Ward-by-Ward Bike Infrastructure Statistics:

Ward  Total_Parking_Spots  Total_Parking_Capacity  Total_Hubs  Total_Hub_Racks  Uncovered_Parking  Uncovered_Hubs  Bikes_Per_Capita_2025  Bikes_Per_Sq_Km  Connectivity_Score  Comfortable_Route_Coverage  Total_Route_Length_km
   1                  203                  3145.0          92             1082                  1               1               0.112482       275.817441           52.837347                    1.975049               0.000380
  10                    1                     0.0           0                0                  0               0               0.000000         0.000000           20.000102                    0.000000               0.000508
  11                    8                    54.0           0                0                  2               0               0.001437         0.274392           15.146607                    0.000000               0.000834
  12                   22                   193.0    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>